In [1]:
%run parse_xml.py

In [2]:
xml_df = parse_XML("paraphrases_mod.xml", ["id", "id_1", "id_2", "text_1", "text_2", "jaccard", "class"])
xml_df.head(3)

,id,id_1,id_2,text_1,text_2,jaccard,class
0,1,201,8159,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0.65,0
1,2,202,8158,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0.5,0
2,3,273,8167,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0.611429,0


In [3]:
xml_df_fltr = xml_df.loc[xml_df['class'].isin(['-1', '1'])]
xml_df_fltr.shape

(4270, 7)

In [13]:
for ind in range(xml_df_fltr.shape[0]):
    if xml_df_fltr.iloc[ind]['class'] == '-1':
        xml_df_fltr.iloc[ind]['class'] = '0'

In [15]:
import zipfile
import gensim 
with zipfile.ZipFile('ru_news_model.zip', 'r') as archive:
    stream = archive.open('model.bin')
    model_w2v_rv = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

In [16]:
import text_processing_rv
%run text_processing_rv.py
import os

# URL of the UDPipe model
udpipe_model_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
udpipe_filename = udpipe_model_url.split('/')[-1]

if not os.path.isfile(udpipe_filename):
    print('UDPipe model not found. Downloading...', file=sys.stderr)
    wget.download(udpipe_model_url)

print('\nLoading the model...', file=sys.stderr)
model = Model.load(udpipe_filename)
process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')



Loading the model...


In [17]:
process(process_pipeline, text=unify_sym('Владимир Путин принял указ о новостях.'.strip()))

['владимир::путин_PROPN ',
 'принимать_VERB',
 'указ_NOUN',
 'о_ADP',
 'новость_NOUN']

In [18]:
mapping_rv_names = dict()
with open("mapping_rv.txt", encoding="utf-8") as file:
    for line in file:
        values = line.split(' ')
#         print(values)
        mapping_rv_names[values[0]] = values[1].strip('\n')
        
# print(mapping_rv_names)     

In [19]:
from stop_words import get_stop_words

stop_words = get_stop_words('russian')
stop_words.append(['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 
                   'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 
                   'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 
                   'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 
                   'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 
                   'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 
                   'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 
                   'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 
                   'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впрочем', 'хорошо', 'свою', 'этой', 'перед', 
                   'иногда', 'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 'всегда', 'конечно', 'всю', 'между'])

In [20]:
udp_sentence_one = []
udp_sentence_two = []
udp_set_words = set([])

for sent in xml_df_fltr['text_1']:   
    sent_coded = [word.rstrip() for word in process(process_pipeline, text=unify_sym(sent.strip()))] # trim strange spases in PROPN
    sent_coded_upd = []
    for word in sent_coded:
        if word in mapping_rv_names.keys():
            sent_coded_upd.append(mapping_rv_names[word])
        elif word[-4:] not in ['_ADP', '_DET', '_SYM'] and word[-5:] not in ['_PART', '_PRON', 
                            'CCONJ', 'SCONJ'] and word[:word.index('_')] not in stop_words:
            sent_coded_upd.append(word.lstrip('-')) # были слова, начинавшиеся с '-'
    udp_sentence_one.append(sent_coded_upd) 
    for word in sent_coded_upd:
        udp_set_words.add(word)
        
for sent in xml_df_fltr['text_2']:   
    sent_coded = [word.rstrip() for word in process(process_pipeline, text=unify_sym(sent.strip()))] # trim strange spases in PROPN
    sent_coded_upd = []
    for word in sent_coded:
        if word in mapping_rv_names.keys():
            sent_coded_upd.append(mapping_rv_names[word])
        elif word[-4:] not in ['_ADP', '_DET', '_SYM'] and word[-5:] not in ['_PART', '_PRON', 
                            'CCONJ', 'SCONJ'] and word[:word.index('_')] not in stop_words:
            sent_coded_upd.append(word.lstrip('-'))
    udp_sentence_two.append(sent_coded_upd) 
    for word in sent_coded_upd:
        udp_set_words.add(word)

In [21]:
import pickle

# Open the file in write binary mode
with open('sentence_one_rusvec_processed_2.txt', 'wb') as F:
    # Dump the list to file
    pickle.dump(udp_sentence_one, F)

# Open the file in write binary mode
with open('sentence_two_rusvec_processed_2.txt', 'wb') as F:
    # Dump the list to file
    pickle.dump(udp_sentence_two, F)
    
# Open the file in write binary mode
with open('unq_words_rusvec_processed_2.txt', 'wb') as F:
    # Dump the list to file
    pickle.dump(udp_set_words, F)

In [22]:
intersect_mydata_rusvect = set.intersection(set(model_w2v_rv.vocab.keys()), udp_set_words)
print("Количество пересечений ", len(intersect_mydata_rusvect))

difference = udp_set_words - set(model_w2v_rv.vocab.keys())
print("Количество слов, которых нет в модели ", len(difference))

Количество пересечений  8212
Количество слов, котрых нет в модели  437


In [23]:
global_count = 0
ids_to_take = [i for i in range(len(udp_sentence_one))]

for sent_1, sent_2, i in zip(udp_sentence_one, udp_sentence_two, range(len(udp_sentence_one))):
    count = 0
    for word in sent_1:
        if word in difference:
            count += 1
    for word in sent_2:
        if word in difference:
            count += 1
    if count > 0:
#         print(sent_1, sent_2)
        global_count += 1
        ids_to_take.remove(i)
        
print("Количество пар предложений, выпадающие из-за отсутствия слова в модели ", global_count)
print("Общее количество пар ", len(udp_sentence_one))

Количество пар предложений, выпадающие изза отсутствия слова в модели  833
Общее количество пар  4270


In [24]:
xml_df_fltr_2 = xml_df_fltr.loc[xml_df_fltr.index[ids_to_take], :]

In [25]:
xml_df_fltr_2.shape

(3437, 7)

In [26]:
xml_df_fltr_2.groupby('class').size()

class
0    2011
1    1426
dtype: int64

In [27]:
xml_df_fltr_2_ones = xml_df_fltr_2.loc[xml_df_fltr_2['class'].isin(['1'])]
xml_df_fltr_2_ones.shape

(1426, 7)

In [28]:
xml_df_fltr_2_zeros = xml_df_fltr_2.loc[xml_df_fltr_2['class'].isin(['0'])]
xml_df_fltr_2_zeros.shape

(2011, 7)

In [29]:
xml_df_fltr_2_zeros_ = xml_df_fltr_2_zeros[0:1426]
xml_df_fltr_2_zeros_.index = xml_df_fltr_2_ones.index

In [30]:
xml_df_fltr_3 = pd.concat([xml_df_fltr_2_ones, xml_df_fltr_2_zeros_], axis= 0)
xml_df_fltr_3.shape

(2852, 7)

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
X_train, X_test, y_train, y_test = train_test_split(
     xml_df_fltr_3, xml_df_fltr_3, test_size=0.33, random_state=42)

In [34]:
X_test.groupby(['class']).size()

class
0    454
1    488
dtype: int64

In [35]:
X_train.groupby(['class']).size()

class
0    972
1    938
dtype: int64

In [36]:
X_test[['class', 'text_1', 'text_2', 'id']].to_csv('test_PP.tsv', sep = '\t', header = None)
X_train[['class', 'text_1', 'text_2', 'id']].to_csv('train_PP.tsv', sep = '\t', header = None)

In [37]:
sum(j**2 for j in model_w2v_rv['цб_PROPN'] / sum([i**2 for i in model_w2v_rv['цб_PROPN']])**(1/2))

0.9999999948601688

In [38]:
with open("wordvec_rv_3.txt", "w") as file:
    for word in intersect_mydata_rusvect:
        vect_norm = model_w2v_rv[word] / ((sum(i**2 for i in model_w2v_rv[word]))**(1/2))
        line_list = [word] + vect_norm.tolist()
        file.write(' '.join(str(item) for item in line_list) + '\n')